# Tutorial Outline

**Tutorial contents**:
- Adding new yield tables
- Choosing element set
- Training of a neural network
- Running MCMC analysis
- Computing Bayes/LOO-CV scores

The above are based on the Philcox & Rybizki (2017) paper which should be cited when using this code. This is based on the $\mathit{Chempy}$ software, described in Rybizki et al. (2017, arXiv:1702.08729) and full tutorials for this can be found at https://github.com/jan-rybizki/Chempy/tree/master/tutorials.

** Requirements**:
Before running this tutorial, the $\mathit{ChempyScoring}$ code and its dependencies must be installed (https://github.com/oliverphilcox/ChempyScoring/blob/master/requirements.txt)

The authors Oliver Philcox (ohep2@cam.ac.uk) and Jan Rybizki (rybizki@mpia.de) are happy to assist with any problems which may arise

## Step 1: Load Yield Tables - ADD TABLE

First we must load in the Nucleosynthetic yield table to be tested. Here we will test the SN2 net yields of Frischknecht et al. (2016, arXiv:1511.05730). These include s-process elements for stars of mass 15-40Msun, with rotation. Here we implement the yield tables for standard rotation and differing metallicities.

The yield tables provide data for masses 15,20,25,40 Msun, metalicities of solar (0.0134), 0.001, 1e-5 and 1e-7 and differing stellar rotation speeds. Here we only use solar, 0.001 and 1e-5 metallicities and standard rotations, since only these have data for all masses. 

To add this into *Chempy* we add a `Frischknecht16_net` function to the `SN2_feedback()` class in `yields.py` as shown below:

In [ ]:
## NB: The Frischknecht16 definition should be inserted into the yields.py file

from Chempy import localpath # For file locations
import numpy as np

class SN2_feedback(object):
    def __init__(self):   
        """
        This is the object that holds the feedback table for SN2 stars.
                The different methods load different tables from the literature. They are in the input/yields/ folder.
        """

    def Frischknecht16_net(self):
        """SN2 yields from Frischknecht et al. 2016. These are implemented for masses of 15-40Msun, for rotating stars.
        Yields from stars with 'normal' rotations are used here.
        These are net yields automatically, so no conversions need to be made
        """
        import numpy.lib.recfunctions as rcfuncs
        import os

        # Define metallicites 
        self.metallicities = [0.0134,1e-3,1e-5] # First is solar value

        # Define masses
        self.masses=  np.array((15,20,25,40))

        # Load yield table dictionary in correct format from npy file if it exists
        saved_yields = localpath+'input/yields/Frischknecht16_net.npy'
        if os.path.exists(saved_yields):
            self.table = np.load(saved_yields).item()

        else:
            # If not, create yield table from .txt file

            # Define data types
            dt = np.dtype('U8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8,f8')

            # Initialise yield table
            yield_table = {}


            # Import full table with correct rows and data-types
            z = np.genfromtxt(localpath+'input/yields/Frischknecht16/yields_total.txt',skip_header=62,dtype=dt)

            # Define isotope indexing. For radioactive isotopes with half-lives << Chempy time_step they are assigned to their daughter element
            # NB: we only use elements up to Ge here, as in the paper
            indexing={}
            indexing['H']=['p','d']
            indexing['He'] = ['he3','he4']
            indexing['Li'] = ['li6','li7']
            indexing['Be']  = ['be9']
            indexing['B']  = ['b10','b11']
            indexing['C']  = ['c12','c13']
            indexing['N']  = ['n14','n15']
            indexing['O']  = ['o16','o17','o18']
            indexing['F']  = ['f19']
            indexing['Ne']  = ['ne20','ne21','ne22']
            indexing['Na']  = ['na23']
            indexing['Mg']  = ['mg24','mg25','mg26','al26']
            indexing['Al']  = ['al27']
            indexing['Si']  = ['si28','si29','si30']
            indexing['P']  = ['p31']
            indexing['S']  = ['s32','s33','s34','s36']
            indexing['Cl']  = ['cl35','cl37']
            indexing['Ar']  = ['ar36','ar38','ar40']
            indexing['K']  = ['k39','k41']
            indexing['Ca']  = ['ca40','ca42','ca43','ca44','ca46','ca48']
            indexing['Sc']  = ['sc45']
            indexing['Ti']  = ['ti46','ti47','ti48','ti49','ti50']
            indexing['V']  = ['v50','v51']
            indexing['Cr']  = ['cr50','cr52','cr53','cr54']
            indexing['Mn']  = ['mn55']
            indexing['Fe']  = ['fe54', 'fe56','fe57','fe58']
            indexing['Co']  = ['fe60', 'co59']
            indexing['Ni']  = ['ni58','ni60','ni61','ni62','ni64']
            indexing['Cu']  = ['cu63','cu65']
            indexing['Zn']  = ['zn64','zn66','zn67','zn68','zn70']
            indexing['Ga']  = ['ga69','ga71']
            indexing['Ge']  = ['ge70','ge72','ge73','ge74','ge76']

            # Define indexed elements 
            self.elements = list(indexing.keys())

            # Create model dictionary indexed by metallicity, giving relevant model number for each choice of mass
            # See Frischknecht info_yields.txt file for model information
            model_dict = {}
            model_dict[0.0134] = [2,8,14,27]
            model_dict[1e-3]=[4,10,16,28]
            model_dict[1e-5]=[6,12,18,29]

            # Import list of remnant masses for each model (from row 32-60, column 6 of .txt file) 
            # NB: these are in solar masses
            rem_mass_table = np.loadtxt(localpath+'input/yields/Frischknecht16/yields_total.txt',skiprows=31,usecols=6)[:29]

            # Create one subtable for each metallicity 
            for metallicity in self.metallicities:
                additional_keys = ['Mass', 'mass_in_remnants','unprocessed_mass_in_winds'] # List of keys for table
                names = additional_keys + self.elements

                # Initialise table and arrays   
                base = np.zeros(len(self.masses))
                list_of_arrays = []
                for i in range(len(names)):
                    list_of_arrays.append(base)
                yield_subtable = np.core.records.fromarrays(list_of_arrays,names=names)
                mass_in_remnants = np.zeros(len(self.masses))
                total_mass_fraction = np.zeros(len(self.masses))
                element_mass = np.zeros(len(self.masses))

                # Add masses to table
                yield_subtable['Mass'] = self.masses


                # Extract remnant masses (in solar masses) for each model:
                for mass_index,model_index in enumerate(model_dict[metallicity]):
                    mass_in_remnants[mass_index] = rem_mass_table[model_index-1] 

               # Iterate over all elements 
                for element in self.elements:
                    element_mass = np.zeros(len(self.masses))
                    for isotope in indexing[element]: # Iterate over isotopes of each element
                        for mass_index,model_index in enumerate(model_dict[metallicity]): # Iterate over masses 
                            for row in z: # Find required row in table 
                                if row[0] == isotope:
                                    element_mass[mass_index]+=row[model_index] # Compute cumulative mass for all isotopes
                    yield_subtable[element]=element_mass # Add entry to subtable

                all_fractions = [row[model_index] for row in z] # This lists all elements (not just up to Ge)
                total_mass_fraction[mass_index] = np.sum(all_fractions) # Compute total net mass fraction (sums to approximately 0)

                # Add fields for remnant mass (now as a mass fraction) and unprocessed mass fraction
                yield_subtable['mass_in_remnants']=np.divide(mass_in_remnants,self.masses)                    
                yield_subtable['unprocessed_mass_in_winds'] = 1.-(yield_subtable['mass_in_remnants']+total_mass_fraction) # This is all mass not from yields/remnants

                # Add subtable to full table
                yield_table[metallicity]=yield_subtable

            # Define final yield table for output
            self.table = yield_table

            # Save yield table to avoid reloading each time
            np.save(saved_yields,self.table)


We can now test the new yield table (using Ca as an example element):

In [2]:
# Define correct yield table
from Chempy.wrapper import SN2_feedback
basic_sn2 = SN2_feedback()
getattr(basic_sn2, 'Frischknecht16_net')()

print("Ca Yields")
for metallicity in basic_sn2.metallicities:
    print("\n Metallicity = %.2e" %(metallicity))
    for i in range(len(basic_sn2.masses)):
        print("Mass = %d, Yield  = %.6e" %(basic_sn2.masses[i],basic_sn2.table[metallicity]['Ca'][i]))


Ca Yields

 Metallicity = 1.34e-02
Mass = 15, Yield  = -4.661234e-05
Mass = 20, Yield  = -1.013723e-04
Mass = 25, Yield  = -1.498169e-04
Mass = 40, Yield  = -3.603430e-04

 Metallicity = 1.00e-03
Mass = 15, Yield  = -2.467764e-06
Mass = 20, Yield  = -4.861126e-06
Mass = 25, Yield  = -8.312735e-06
Mass = 40, Yield  = -1.565401e-05

 Metallicity = 1.00e-05
Mass = 15, Yield  = -2.234603e-08
Mass = 20, Yield  = -4.682224e-08
Mass = 25, Yield  = -6.924936e-08
Mass = 40, Yield  = -1.481142e-07


In [3]:
basic_sn2.table[1e-3]['B']

array([ -8.71712663e-10,  -1.17302861e-09,  -1.90546586e-09,
        -2.32895755e-09])

## Step 2: Choice of Elements

We are free to use any set of chemical elements in this analysis. The set should be chosen to match the simulation. 28 elements up to Ge (excluding Li, B, Be) and the IllustrisTNG elements (Pillepich et al. 2017) were used in the Philcox & Rybizki 2017 paper.

To select the required elements we modify the `Chempy/parameter.py` file.

Both `elements_to_trace` **and** `initial_neural_names` must be changed here.


*`elements_to_trace` contains elements in the proto-solar data-file, including B, Be, Li and H which are not predicted directly by the neural network. `initial_neural_names` is the elements predicted by the network only (as [X/Fe] or [Fe/H] abundances.*

In addition, if extra elements are added, it should be checked that they are predicted by the yield tables and feature in the `Chempy/input/stars/proto_sun_all.npy` observational data-set.


In [1]:
## Modify these lines in Chempy/parameter.py

# This field should contain all required elements  (and B,Be,Li,H) in alphabetical order
elements_to_trace = ['Al', 'Ar', 'B', 'Be', 'C', 'Ca', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'H', 'He', 'K', 'Li', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Sc', 'Si', 'Ti', 'V', 'Zn']

# This field contains names of elements predicted by neural network
initial_neural_names = ['Al', 'Ar', 'C', 'Ca', 'Cl', 'Co', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'Ge', 'He', 'K', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni', 'O', 'P', 'S', 'Sc', 'Si', 'Ti', 'V', 'Zn']


## Step 3: Create Neural Network Dataset

Now that the yield set has been implemented, we must next create a training data-set for the neural network.

Firstly it is important to change the `parameter.py` file such that *Chempy* uses the correct yields. Here we add the new SN2 yield table name and set *Chempy* to use it by default. 

In [2]:
## Modify these lines in Chempy/parameter.py to add new yield set
yield_table_name_sn2_list = ['chieffi04','Nugrid','Nomoto2013','Portinari', 'chieffi04_net', 'Nomoto2013_net','NuGrid_net','West17_net','TNG_net']#'Frischknecht16_net'
yield_table_name_sn2_index = 7
yield_table_name_sn2 = yield_table_name_sn2_list[yield_table_name_sn2_index]


We can test this as follows:

In [1]:
# Load parameter file
from Chempy.parameter import ModelParameters
a = ModelParameters()

# Print new SN2 yield table name
print(a.yield_table_name_sn2)

West17_net


We must also set the list of parameters to optimize over to include only the 5 free *Chempy* parameters (i.e. not $\beta$). This will be changed later in the analysis, but must be done at this point, else the `training_data()` routine will fail. 

**Here we can also change the free parameters **

For compatibility reasons $\beta$ (when later added) is included in the SSP_parameters definition.


In [4]:
# Modify these lines in Chempy/parameter.py file

SSP_parameters =  [-2.29,-2.75]
SSP_parameters_to_optimize = ['high_mass_slope','log10_N_0']
assert len(SSP_parameters) == len(SSP_parameters_to_optimize)

ISM_parameters =  [-0.3,0.55,0.5]
ISM_parameters_to_optimize = ['log10_starformation_efficiency', 'log10_sfr_scale', 'outflow_feedback_fraction']
assert len(ISM_parameters) == len(ISM_parameters_to_optimize)



We must also turn OFF the neural network predictions:

In [2]:
# In Chempy/parameter.py
UseNeural=False

# To test
a.UseNeural

False

Data-sets can be created using the `Chempy.neural` module as follows and are saved in the `Neural/` directory. 

This uses multiprocessing to create a training data-set using 10 values of each of the 5 free *Chempy* parameters. (This value can be changed using `training_size` in `parameter.py`). These are written to file as `Neural/training_abundances.npy` (abundance output) and `Neural/training_norm_grid.npy` (normalised input)

In [1]:
from Chempy.neural import training_data
#training_data()

The above was run on a 64-core machine, taking 45 minutes. 

## Step 4: Train Neural Network

Next we must train the neural network using the previously constructed data-sets. This can be simply done with the `Chempy/neural.py` `create_network()` function. This creates and trains a 30-neuron network over 1000 training epochs, using a learning rate of 0.007 by default (optimised via a validation data-set).

In [4]:
from Chempy.neural import create_network
#create_network(Plot=True)

The above was run on an 8-core machine taking 20 minutes. The trained network is saved as `Neural/neural_model.npz`. A loss plot is also produced, showing the network loss function against training epoch (if `Plot=True`).

Using `Chempy/neural.py`'s `neural_output()` function we may simulate the output of *Chempy* for any set of input parameters;

In [5]:
from Chempy.parameter import ModelParameters
a = ModelParameters()

from Chempy.neural import neural_output

# Here we compute the neural network predictions for the prior values of the free parameters (a.p0) 
output = neural_output(a.p0)

print("Element \t Predicted abundance")
print("------------------------------------")
for i in range(len(output)):
    print(a.initial_neural_names[i],"\t\t",output[i])

Element 	 Predicted abundance
------------------------------------
Al 		 -0.316759824738
Ar 		 0.312959080369
C 		 -0.266671754526
Ca 		 -0.525946020995
Cl 		 -0.101032700348
Co 		 -0.363575246299
Cr 		 -0.305827662797
Cu 		 0.0580966357488


## Step 5: Run MCMC analysis

We can now run MCMC on the yield set and produce optimal parameters and a corner plot. This is done as shown below, using a trained neural network.

Whichever neural network is in the `Neural/` directory will be used

In [1]:
# This is a wrapper for MCMC (with initial global optimisation to speed up convergence)
from Chempy.wrapper import single_star_optimization
single_star_optimization() # NB: if not using a neural network, multi_star_optimization() should be used

['Proto-sun_all']
first minimization for each star separately took:  0 seconds
step  1 of  1000
1.73569619634 1.7353424002
calculation so far took 1.6636462211608887  seconds
step  2 of  1000
1.73569619634 1.73446346283
calculation so far took 1.9948980808258057  seconds
step  3 of  1000
1.73569619634 1.73111670318
calculation so far took 2.3175220489501953  seconds
step  4 of  1000
1.73569619634 1.72968438599
calculation so far took 2.7558858394622803  seconds
step  5 of  1000
1.73569619634 1.72378296961
calculation so far took 3.1382975578308105  seconds
step  6 of  1000
1.73569619634 1.70758419918
calculation so far took 3.51373291015625  seconds
step  7 of  1000
1.73569619634 1.69546069415
calculation so far took 3.8425889015197754  seconds
step  8 of  1000
1.73569619634 1.65553515173
calculation so far took 4.18914532661438  seconds
step  9 of  1000
1.73569619634 1.6015698814
calculation so far took 4.543027877807617  seconds
step  10 of  1000
1.73569619634 1.46826507008
calculati

1.73569619634 0.874550862019
calculation so far took 29.829168558120728  seconds
step  85 of  1000
1.73569619634 0.747898520289
calculation so far took 30.166311502456665  seconds
step  86 of  1000
1.73569619634 0.500359910389
calculation so far took 30.47402048110962  seconds
step  87 of  1000
1.73569619634 0.3047447488
calculation so far took 30.782214641571045  seconds
step  88 of  1000
1.73569619634 0.324448250287
calculation so far took 31.093920946121216  seconds
step  89 of  1000
1.73569619634 0.344398696378
calculation so far took 31.416279077529907  seconds
step  90 of  1000
1.73569619634 0.416643162612
calculation so far took 31.729092597961426  seconds
step  91 of  1000
1.73569619634 0.591688326557
calculation so far took 32.045167446136475  seconds
step  92 of  1000
1.73569619634 0.71952873252
calculation so far took 32.35832142829895  seconds
step  93 of  1000
1.73569619634 0.673198820489
calculation so far took 32.67647385597229  seconds
step  94 of  1000
1.73569619634 0.

1.73569619634 0.892286368603
calculation so far took 57.78285574913025  seconds
step  168 of  1000
1.73569619634 0.819012153616
calculation so far took 58.12380337715149  seconds
step  169 of  1000
1.73569619634 0.755968735473
calculation so far took 58.46123647689819  seconds
step  170 of  1000
1.73569619634 0.742560263055
calculation so far took 58.78768181800842  seconds
step  171 of  1000
1.73569619634 0.94297425987
calculation so far took 59.149317264556885  seconds
step  172 of  1000
1.73569619634 1.23469731234
calculation so far took 59.49305820465088  seconds
step  173 of  1000
1.73569619634 1.09166600779
calculation so far took 59.83216738700867  seconds
step  174 of  1000
1.73569619634 0.892427442737
calculation so far took 60.181599617004395  seconds
step  175 of  1000
1.73569619634 0.813513183578
calculation so far took 60.51658749580383  seconds
step  176 of  1000
1.73569619634 0.637795377184
calculation so far took 60.84281611442566  seconds
step  177 of  1000
1.735696196

1.73569619634 1.00829058801
calculation so far took 84.81325054168701  seconds
step  252 of  1000
1.73569619634 0.934569381939
calculation so far took 85.14925003051758  seconds
step  253 of  1000
1.73569619634 1.0997155267
calculation so far took 85.48931050300598  seconds
step  254 of  1000
1.73569619634 0.913683235668
calculation so far took 85.80446076393127  seconds
step  255 of  1000
1.73569619634 1.01269284634
calculation so far took 86.13122391700745  seconds
step  256 of  1000
1.73569619634 1.13767169983
calculation so far took 86.44597792625427  seconds
step  257 of  1000
1.73569619634 1.27026217753
calculation so far took 86.7755115032196  seconds
step  258 of  1000
1.73569619634 1.29070244067
calculation so far took 87.09938406944275  seconds
step  259 of  1000
1.73569619634 1.04131671026
calculation so far took 87.46312880516052  seconds
step  260 of  1000
1.73569619634 1.06860324964
calculation so far took 87.83285307884216  seconds
step  261 of  1000
1.73569619634 1.2171

[array([-2.22414284, -3.34377459, -0.28479125,  0.51975264,  0.45931538]),
 'initial minimization']

In [ ]:
from Chempy.plot_mcmc import restructure_chain,plot_mcmc_chain_with_prior
restructure_chain('mcmc/') # Restructure the MCMC chain

# Compute best parameter values


START HERE

In [ ]:
# Now create the corner plot (saved as mcmc/parameter_space_sorted.png)
plot_mcmc_chain_with_prior('BestMCMC/',use_prior=True,only_first_star=False,plot_true_parameters=False,plot_only_SSP_parameter=False)

Compute Bayes + LOO-CV Scores as a function of $\beta$

We are now ready to compute both the Bayes and LOO-CV scores for the network. Before computation, we instruct *Chempy* to use the trained neural network which is done by altering the `Chempy/parameter.py` file:

In [2]:
## Modify these lines in Chempy/parameter.py
UseNeural = True 

# To test if this has worked:
a.UseNeural

True

## Step 5: Compute Overall Scores